In [ ]:
import asyncio
import importlib
from pathlib import Path
from pprint import pprint

import instructor
import pandas as pd
from openai import OpenAI

from cuery import utils

importlib.reload(utils)

<module 'cellmagic.utils' from '/Users/thomas/code/cellmagic/src/cellmagic/utils.py'>

# Bugs

In [ ]:
import instructor
from pydantic import BaseModel
from openai import AsyncOpenAI, OpenAI

# Enables `response_model` in `create` method
client = instructor.from_openai(AsyncOpenAI())


class Person(BaseModel):
    name: str
    age: int


async def extract_person(text: str) -> Person:
    return await client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": text},
        ],
        response_model=Person,
    )


result = await extract_person("My name is John Doe and I am 30 years old.")
result

Person(name='John Doe', age=30)

# Litellm

In [16]:
from litellm import completion

utils.set_litellm_api_keys()
response = completion(
    model="openai/gpt-4o",
    messages=[{"content": "Hello, how are you?", "role": "user"}],
)
print(response.choices[0].message.content)

AttributeError: module 'cellmagic.utils' has no attribute 'set_litellm_api_keys'

In [ ]:
utils.set_litellm_api_keys()
response = completion(
    model="perplexity/sonar-pro",
    messages=[{"content": "Hello, how are you?", "role": "user"}],
)
print(response.choices[0].message.content)

Hello! Thanks for asking—I'm here and ready to help you with any questions or information you need. If you're inspired by the cheerful greeting from Super Simple Songs, I'd say:

"I'm good! I'm great! I'm wonderful!"[1][3] 

How can I assist you today?


# Instructor

In [ ]:
import instructor
from litellm import completion
from pydantic import BaseModel


class ExtractUser(BaseModel):
    name: str
    age: int


client = instructor.from_litellm(completion)

resp = client.chat.completions.create(
    model="openai/gpt-4o",
    messages=[
        {
            "role": "user",
            "content": "Extract Users from the following text: Jason is 25 years old.",
        }
    ],
    response_model=ExtractUser,
)

assert isinstance(resp, ExtractUser)
assert resp.name == "Jason"
assert resp.age == 25

In [ ]:
resp

ExtractUser(name='Jason', age=25)

# Tasks

In [2]:
GDRIVE = Path("~/Google Drive/Shared drives/Solutions").expanduser()
DATA_DIR = GDRIVE / "Research/future_of_work/inputs/ine_dirce_aggregated_by_activity.csv"

In [7]:
data = pd.read_csv(DATA_DIR)
display(data)

n_examples = 5
exs = data.loc[:n_examples, ["Division", "Actividad principal"]]
exs = exs.values.tolist()
exs

context_df = (
    data[["Division", "Actividad principal"]]
    .iloc[:3]
    .rename(columns={"Division": "division", "Actividad principal": "actividad"})
)

context_dict = {
    "division": context_df["division"].tolist(),
    "actividad": context_df["actividad"].to_list(),
}

context_records = context_df.to_dict(orient="records")


,Division,Actividad principal,Estimated_Employees_2024,Median_YoY_Growth_pct
0,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4
1,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2
2,43 Actividades de construcción especializada,"432 Instalaciones eléctricas, de fontanería y ...",303430,0.5
3,45 Venta y reparación de vehículos de motor y ...,451 Venta de vehículos de motor,102172,-0.5
4,46 Comercio al por mayor e intermediarios del ...,463 Comercio al por mayor de productos aliment...,325970,-0.9
5,46 Comercio al por mayor e intermediarios del ...,464 Comercio al por mayor de artículos de uso ...,228324,-2.0
6,"47 Comercio al por menor, excepto de vehículos...",471 Comercio al por menor en establecimientos ...,233708,1.3
7,"47 Comercio al por menor, excepto de vehículos...",477 Comercio al por menor de otros artículos e...,242341,-1.3
8,49 Transporte terrestre y por tubería,493 Otro transporte terrestre de pasajeros,142413,0.5
9,49 Transporte terrestre y por tubería,494 Transporte de mercancías por carretera y s...,273002,-1.5


In [ ]:
from cuery import prompt, utils

utils.set_api_keys()

from cuery.work import tasks

importlib.reload(utils)
importlib.reload(prompt)
importlib.reload(tasks)

client = instructor.from_provider("openai/gpt-3.5-turbo", async_client=True)

In [9]:
jobs = await prompt.call(
    client,
    model="gpt-3.5-turbo",
    prompt=tasks.PROMPTS["dirce_jobs"],
    context={"division": exs[0][0], "actividad": exs[1][1]},
    response_model=tasks.Jobs,
)

print(jobs)

jobs=[Job(name='Técnico en automatización industrial', description='Responsable de diseñar e implementar sistemas automatizados en procesos industriales.', automation_potential=7, reason='Repetitive tasks with clear rules and patterns.'), Job(name='Diseñador de productos', description='Crea conceptos y diseños para nuevos productos en la industria de fabricación de componentes para vehículos.', automation_potential=5, reason='Potential for computer-aided design to automate tasks.'), Job(name='Ingeniero de Control de Calidad', description='Responsable de garantizar los estándares de calidad en la fabricación de componentes para vehículos.', automation_potential=6, reason='Potential for AI to analyze quality metrics.')]


In [10]:
jobs = await prompt.iter_calls(
    client,
    model=None,  # "gpt-3.5-turbo",
    prompt=tasks.PROMPTS["dirce_jobs"],
    context=context_records,
    response_model=tasks.Jobs,
)
pprint(jobs)

[Jobs(jobs=[Job(name='Técnico de procesamiento de carne', description='Responsable del procesamiento y conservación de carne en la industria cárnica.', automation_potential=7, reason='Tareas repetitivas y predecibles.'), Job(name='Operario de línea de producción de carne', description='Encargado de la producción en cadena de productos cárnicos.', automation_potential=8, reason='Actividades que pueden ser estandarizadas.'), Job(name='Inspector de calidad de productos cárnicos', description='Supervisa la calidad de los productos cárnicos en diferentes etapas de producción.', automation_potential=5, reason='Necesidad de análisis visual y juicio.')]),
 Jobs(jobs=[Job(name='Ingeniero de Automatización Industrial', description='Encargado de diseñar y desarrollar sistemas de automatización industrial en el sector de fabricación de vehículos y componentes.', automation_potential=8, reason='Tecnología avanzada y programación automatizable.'), Job(name='Operador de Máquinas CNC', description='Tr

In [ ]:
jobs = await prompt.gather_calls(
    client,
    model="gpt-3.5-turbo",
    prompt=tasks.PROMPTS["dirce_jobs"],
    context=context_records,
    response_model=tasks.Jobs,
)
pprint(jobs)

[Jobs(jobs=[Job(name='Técnico en procesamiento de carne', description='Supervisión y control de las actividades de procesado de carne', automation_potential=7, reason='Tareas repetitivas y basadas en reglas'), Job(name='Operario de maquinaria de procesado de carne', description='Operación y mantenimiento de maquinaria de procesado de carne', automation_potential=8, reason='Operaciones predecibles y repetitivas'), Job(name='Ingeniero de calidad de productos cárnicos', description='Garantizar la calidad de los productos cárnicos', automation_potential=6, reason='Análisis predictivo y control de calidad')]),
 Jobs(jobs=[Job(name='Ingeniero de Automatización', description='Responsable de diseñar e implementar sistemas y procesos de automatización en la fabricación de vehículos.', automation_potential=8, reason='Automatización de tareas repetitivas y programables.'), Job(name='Técnico en Robótica', description='Encargado de mantener y optimizar los sistemas de robots utilizados en la fabric

In [14]:
jobs = await prompt.gather_calls(
    client,
    model="gpt-3.5-turbo",
    prompt=tasks.PROMPTS["dirce_jobs"],
    context=context_df,
    response_model=tasks.Jobs,
)
pprint(jobs)

[Jobs(jobs=[Job(name='Técnico en procesamiento de carne', description='Responsable del procesamiento y conservación de carne, supervisión de la producción de productos cárnicos.', automation_potential=7, reason='Repetitive tasks in production process can be automated.'), Job(name='Operario de línea de producción de carne', description='Encargado de la operación de la línea de producción de productos cárnicos.', automation_potential=6, reason='Routine tasks in production line can be automated.'), Job(name='Inspector de calidad de productos cárnicos', description='Responsable de verificar la calidad de los productos cárnicos producidos.', automation_potential=5, reason='Quality assurance processes can be automated with AI technologies.')]),
 Jobs(jobs=[Job(name='Ingeniero de control de calidad', description='Responsable de asegurar la calidad de los componentes fabricados para vehículos de motor', automation_potential=7, reason='Rutinario y basado en reglas predefinidas'), Job(name='Oper

In [11]:
%autoawait asyncio
dirce = tasks.DirceJobs
result = await dirce.gather(context=context_df)
result

[Jobs(jobs=[Job(name='Técnico de Procesamiento de Carne', description='Supervisa y optimiza el procesamiento de carne en plantas industriales.', automation_potential=8, reason='Repetitivo y basado en reglas predefinidas'), Job(name='Operario de Producción de Productos Cárnicos', description='Realiza tareas en la línea de producción de productos cárnicos.', automation_potential=7, reason='Tareas repetitivas y predecibles'), Job(name='Inspector de Calidad de Productos Cárnicos', description='Revisa la calidad de los productos cárnicos según estándares establecidos.', automation_potential=6, reason='Basado en evaluación de criterios predefinidos')]),
 Jobs(jobs=[Job(name='Ingeniero de Procesos de Automatización', description='Desarrolla y coordina procesos automatizados en la industria', automation_potential=8, reason='Tareas repetitivas y programables'), Job(name='Técnico en Robótica Industrial', description='Mantenimiento y programación de robots industriales', automation_potential=7, r

In [ ]:
def explode_list_models(models, field, to_pandas=True, **ids_vars):
    results = []
    for i, m in enumerate(models):
        for item in getattr(m, field):
            ids = {k: v[i] for k, v in ids_vars.items()}
            item = ids | dict(item)
            results.append(item)

    if to_pandas:
        return pd.DataFrame.from_records(results)

    return results


explode_list_models(jobss2, "jobs", division=divs, actividad=acts)

name='Técnico en procesamiento de alimentos' description='Responsable del procesamiento y conservación de alimentos, incluida la carne y productos cárnicos. Supervisión de equipos y cumplimiento de normas de calidad.' automation_potential=7 reason='Tareas repetitivas y programables.'


,division,actividad,name,description,automation_potential,reason
0,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,Técnico en procesamiento de alimentos,Responsable del procesamiento y conservación d...,7,Tareas repetitivas y programables.
1,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,Inspector de calidad de productos alimenticios,Encargado de verificar la calidad de los produ...,6,Tareas basadas en estándares predefinidos.
2,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,Operario de envasado de alimentos,"Encargado del envasado de alimentos, especialm...",8,Procesos mecánicos y repetitivos.
3,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",Ingeniero de Automatización,Diseña y desarrolla sistemas automáticos para ...,7,Highly repetitive tasks can be automated.
4,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",Operario de Producción,Realiza tareas repetitivas en línea de producc...,8,Routine tasks can be replaced by machines.
5,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",Técnico de Mantenimiento Industrial,Realiza mantenimiento preventivo y correctivo ...,5,Some maintenance tasks can be automated.
6,43 Actividades de construcción especializada,"432 Instalaciones eléctricas, de fontanería y ...",Electricista de obra,Instalación y mantenimiento de sistemas eléctr...,7,Tareas rutinarias y repetitivas que pueden ser...
7,43 Actividades de construcción especializada,"432 Instalaciones eléctricas, de fontanería y ...",Fontanero de obra,Instalación y mantenimiento de sistemas de fon...,6,Tareas repetitivas y basadas en patrones que p...
8,43 Actividades de construcción especializada,"432 Instalaciones eléctricas, de fontanería y ...",Auxiliar de obra,Ayuda en tareas generales de construcción y ma...,3,Tareas manuales que requieren flexibilidad y a...
9,45 Venta y reparación de vehículos de motor y ...,451 Venta de vehículos de motor,Asesor de ventas de vehículos,Responsable de la venta de vehículos de motor ...,7,High potential for automation with digital sal...
